In [1]:
import pandas as pd
import numpy as np

In [2]:
# import patient data
df = pd.read_csv('dataset/patient_dg.csv')
df

,Changwat E,Odpc,Region,กลุ่มอายุ,Agey,Agey (copy),Datesick,Number of Records,Sick_YEAR_2019
0,Phrae,สคร.1,ภาคเหนือ,15-24,23,23,10/10/2020,1,NaN
1,Chiang Rai,สคร.1,ภาคเหนือ,15-24,24,24,9/5/2020,1,NaN
2,Phitsanulok,สคร.2,ภาคเหนือ,25-34,25,25,5/29/2020,1,NaN
3,Chiang Mai,สคร.1,ภาคเหนือ,15-24,24,24,3/2/2020,1,NaN
4,Chiang Mai,สคร.1,ภาคเหนือ,35-59,35,35,3/13/2020,1,NaN
...,...,...,...,...,...,...,...,...,...
476930,Kanchanaburi,สคร.5,ภาคกลาง,60,66,66,8/2/2016,1,NaN
476931,Phra Nakhon Si Ayutthaya,สคร.4,ภาคกลาง,35-59,40,40,3/10/2016,1,NaN
476932,Suphan Buri,สคร.5,ภาคกลาง,60,71,71,10/21/2016,1,NaN
476933,Samut Prakan,สคร.6,ภาคกลาง,15-24,23,23,2/24/2016,1,NaN


In [3]:
# rename column to appropriate name
df2 = df.rename(columns={'กลุ่มอายุ': 'AgeGroup','Changwat E':'Province'})
df2

,Province,Odpc,Region,AgeGroup,Agey,Agey (copy),Datesick,Number of Records,Sick_YEAR_2019
0,Phrae,สคร.1,ภาคเหนือ,15-24,23,23,10/10/2020,1,NaN
1,Chiang Rai,สคร.1,ภาคเหนือ,15-24,24,24,9/5/2020,1,NaN
2,Phitsanulok,สคร.2,ภาคเหนือ,25-34,25,25,5/29/2020,1,NaN
3,Chiang Mai,สคร.1,ภาคเหนือ,15-24,24,24,3/2/2020,1,NaN
4,Chiang Mai,สคร.1,ภาคเหนือ,35-59,35,35,3/13/2020,1,NaN
...,...,...,...,...,...,...,...,...,...
476930,Kanchanaburi,สคร.5,ภาคกลาง,60,66,66,8/2/2016,1,NaN
476931,Phra Nakhon Si Ayutthaya,สคร.4,ภาคกลาง,35-59,40,40,3/10/2016,1,NaN
476932,Suphan Buri,สคร.5,ภาคกลาง,60,71,71,10/21/2016,1,NaN
476933,Samut Prakan,สคร.6,ภาคกลาง,15-24,23,23,2/24/2016,1,NaN


In [4]:
# count number of patient in each day by/province by agegroup
df3 = df2.groupby(['Datesick','Province','AgeGroup'])['Number of Records'].agg('count').unstack(fill_value=0).stack().reset_index()
df3 = pd.DataFrame(df3).rename(columns={df3.columns[3]: 'patient_count'})
df3

,Datesick,Province,AgeGroup,patient_count
0,1/1/2016,Amnat Charoen,0-4,0
1,1/1/2016,Amnat Charoen,10-14,2
2,1/1/2016,Amnat Charoen,15-24,0
3,1/1/2016,Amnat Charoen,25-34,0
4,1/1/2016,Amnat Charoen,35-59,0
...,...,...,...,...
642910,9/9/2020,Yasothon,15-24,2
642911,9/9/2020,Yasothon,25-34,0
642912,9/9/2020,Yasothon,35-59,0
642913,9/9/2020,Yasothon,5-9,0


In [5]:
# Check grouping algorithm is working correctly 
# at Datesick == 1/1/2016, Changwat E == Amnat Charoen
# There are 2 records of 10-14 --> match with above record
df2.loc[df2['Province']=='Amnat Charoen',['Datesick','Province','AgeGroup']].sort_values('Datesick')

,Datesick,Province,AgeGroup
374234,1/1/2016,Amnat Charoen,10-14
380038,1/1/2016,Amnat Charoen,10-14
371100,1/11/2016,Amnat Charoen,25-34
352765,1/11/2016,Amnat Charoen,25-34
250924,1/11/2020,Amnat Charoen,15-24
...,...,...,...
74615,9/9/2019,Amnat Charoen,15-24
140194,9/9/2019,Amnat Charoen,5-9
448051,9/9/2019,Amnat Charoen,10-14
90518,9/9/2019,Amnat Charoen,5-9


In [6]:
# Applying one hot encoding to age group
agegroup_dummy = pd.get_dummies(df3.AgeGroup, prefix='ag')
df4 = pd.concat([df3, agegroup_dummy], axis=1)
df4

,Datesick,Province,AgeGroup,patient_count,ag_0-4,ag_10-14,ag_15-24,ag_25-34,ag_35-59,ag_5-9,ag_60
0,1/1/2016,Amnat Charoen,0-4,0,1,0,0,0,0,0,0
1,1/1/2016,Amnat Charoen,10-14,2,0,1,0,0,0,0,0
2,1/1/2016,Amnat Charoen,15-24,0,0,0,1,0,0,0,0
3,1/1/2016,Amnat Charoen,25-34,0,0,0,0,1,0,0,0
4,1/1/2016,Amnat Charoen,35-59,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
642910,9/9/2020,Yasothon,15-24,2,0,0,1,0,0,0,0
642911,9/9/2020,Yasothon,25-34,0,0,0,0,1,0,0,0
642912,9/9/2020,Yasothon,35-59,0,0,0,0,0,1,0,0
642913,9/9/2020,Yasothon,5-9,0,0,0,0,0,0,1,0


In [7]:

df4["Datesick"] = pd.to_datetime(df3["Datesick"], format='%m/%d/%Y')
df4['day'] = df4['Datesick'].map(lambda x: x.day)
df4['month'] = df4['Datesick'].map(lambda x: x.month)
df4['year'] = df4['Datesick'].map(lambda x: x.year)
df4

,Datesick,Province,AgeGroup,patient_count,ag_0-4,ag_10-14,ag_15-24,ag_25-34,ag_35-59,ag_5-9,ag_60,day,month,year
0,2016-01-01,Amnat Charoen,0-4,0,1,0,0,0,0,0,0,1,1,2016
1,2016-01-01,Amnat Charoen,10-14,2,0,1,0,0,0,0,0,1,1,2016
2,2016-01-01,Amnat Charoen,15-24,0,0,0,1,0,0,0,0,1,1,2016
3,2016-01-01,Amnat Charoen,25-34,0,0,0,0,1,0,0,0,1,1,2016
4,2016-01-01,Amnat Charoen,35-59,0,0,0,0,0,1,0,0,1,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642910,2020-09-09,Yasothon,15-24,2,0,0,1,0,0,0,0,9,9,2020
642911,2020-09-09,Yasothon,25-34,0,0,0,0,1,0,0,0,9,9,2020
642912,2020-09-09,Yasothon,35-59,0,0,0,0,0,1,0,0,9,9,2020
642913,2020-09-09,Yasothon,5-9,0,0,0,0,0,0,1,0,9,9,2020


In [8]:
# Rearrange column
cols = df4.columns.tolist()
cols = cols[0:3] + cols[-3:] + cols[4:-3] + cols[3:4] # [first 3 col] + [last 3 col (d,m,y)] + [(age group)] + patient count
df5 = df4[cols]
df5

,Datesick,Province,AgeGroup,day,month,year,ag_0-4,ag_10-14,ag_15-24,ag_25-34,ag_35-59,ag_5-9,ag_60,patient_count
0,2016-01-01,Amnat Charoen,0-4,1,1,2016,1,0,0,0,0,0,0,0
1,2016-01-01,Amnat Charoen,10-14,1,1,2016,0,1,0,0,0,0,0,2
2,2016-01-01,Amnat Charoen,15-24,1,1,2016,0,0,1,0,0,0,0,0
3,2016-01-01,Amnat Charoen,25-34,1,1,2016,0,0,0,1,0,0,0,0
4,2016-01-01,Amnat Charoen,35-59,1,1,2016,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642910,2020-09-09,Yasothon,15-24,9,9,2020,0,0,1,0,0,0,0,2
642911,2020-09-09,Yasothon,25-34,9,9,2020,0,0,0,1,0,0,0,0
642912,2020-09-09,Yasothon,35-59,9,9,2020,0,0,0,0,1,0,0,0
642913,2020-09-09,Yasothon,5-9,9,9,2020,0,0,0,0,0,1,0,0


In [15]:
df6 = df5.drop(columns = ['Datesick'])
df6 = df6.drop(columns = ['day'])
df6 = df6.drop(columns = ['month'])
df6

,Province,AgeGroup,year,ag_0-4,ag_10-14,ag_15-24,ag_25-34,ag_35-59,ag_5-9,ag_60,patient_count
0,Amnat Charoen,0-4,2016,1,0,0,0,0,0,0,0
1,Amnat Charoen,10-14,2016,0,1,0,0,0,0,0,2
2,Amnat Charoen,15-24,2016,0,0,1,0,0,0,0,0
3,Amnat Charoen,25-34,2016,0,0,0,1,0,0,0,0
4,Amnat Charoen,35-59,2016,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
642910,Yasothon,15-24,2020,0,0,1,0,0,0,0,2
642911,Yasothon,25-34,2020,0,0,0,1,0,0,0,0
642912,Yasothon,35-59,2020,0,0,0,0,1,0,0,0
642913,Yasothon,5-9,2020,0,0,0,0,0,1,0,0


In [18]:
df6.to_csv('dg_group.csv')